In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
import seaborn as sns
import requests
from data_loading import *
import plotly.graph_objects as go


In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

### base loading and visualizing

In [ ]:
df_form, columns_series, program_columns_ratings, program_columns, form_ratings, form_attends, open_answers = load_form_data()

In [ ]:
df_app = load_app_data()

In [ ]:
df_timetable = load_api_info(df_form.columns)

In [ ]:
app_ratings = prepare_app_ratings(df_app)
app_ratings

In [ ]:
df_form.head()

In [ ]:
program_columns_ratings.head()

In [ ]:
program_columns.head()

In [ ]:
df_timetable.head()

### filtering, keeping only entries in feedback form

In [ ]:
form_attends.head()

In [ ]:
form_ratings

### starting analyzing apps

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
df_app['uživatel'].value_counts().head()

#### preparing app data format for merge with form data

### base EDA

In [ ]:
df_form.head()

In [ ]:
df_form.describe()

In [ ]:
[(i, len(df_form[i].unique())) for i in df_form.columns if len(df_form[i].unique()) < 10]

In [ ]:
df_form.columns.map(lambda i: len(df_form[i].unique())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
df_form.columns.map(lambda i: sum(df_form[i].notnull())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
attend_columns = [i for i in df_form.columns if 'Nezúčastnil(a) jsem se' in df_form[i].values]

In [ ]:
attend_columns[:10]

In [ ]:
len(attend_columns)

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 10])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 20])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100])

In [ ]:
all_attends = df_form[attend_columns].stack()
all_attends[~all_attends.isna()].value_counts().plot.pie()

In [ ]:
all_attends[~all_attends.isna()].value_counts().plot.barh()

In [ ]:
all_attends[(~all_attends.isna()) & ('Nezúčastnil(a) jsem se' != all_attends)].value_counts().plot.barh()

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
def abs_vals(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return '{:d}'.format(absolute)


In [ ]:
feedback_geo = df_form['Bydliště'].value_counts()
feedback_geo.plot.pie(autopct=partial(abs_vals, allvals=feedback_geo))

In [ ]:
#for i in [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100]:
#    plt.figure()
#    feedback_i = df_form[i].value_counts()
#    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i))
#    plt.show()


In [ ]:
[i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]

In [ ]:
cols_to_plot = [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]
cols = 3
fig, axes = plt.subplots(int(np.ceil(len(cols_to_plot) / cols)), cols, figsize=(35, 120))

for i, col in enumerate(cols_to_plot):
    ax = axes[i // cols, i % cols]
    #plt.figure()
    feedback_i = df_form[col].value_counts()
    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i), ax=ax, y=None)
plt.show()

In [ ]:
feedback_times = df_form['Timestamp'].value_counts(sort=False).copy().reindex().resample('1D', how='count')
feedback_times.plot.bar()
plt.title('Počet vyplnění za den, vše červen')
#plt.gca().set_xticklabels([dt.strftime("%#d. %#m.") for dt in feedback_times.index])
plt.gca().set_xticklabels([dt.strftime("%#d.") for dt in feedback_times.index])
plt.xticks(rotation=0)
plt.show()

#### merging with app data

In [ ]:
form_ratings.columns.unique()

In [ ]:
form_attends.T.columns.unique()

In [ ]:
set(form_ratings.columns) - set(form_attends.T.columns)  # these will be dropped

In [ ]:
set(form_attends.T.columns) - set(form_ratings.columns)

### differences between app and form rating fields

In [ ]:
set(form_ratings.columns) - set(app_ratings.columns)

In [ ]:
set(app_ratings.columns) - set(form_ratings.columns)

In [ ]:
app_ratings.head()

In [ ]:
program_ratings, program_attends, program_columns = merge_ratings(form_ratings, app_ratings)

In [ ]:
form_ratings

In [ ]:
program_ratings

In [ ]:
program_attends.T

In [ ]:
def remove_suffix(string):
    return string.replace(info_suffix, '').replace(fun_suffix, '')
form_ratings.groupby(form_ratings.columns.map(remove_suffix), axis=1).max()

In [ ]:
program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max()

In [ ]:
program_attends.head()

In [ ]:
type_to_color = {'přednášky': 'g', 'workshopy': 'purple', 'soutěže': 'yellow', 'doplňkový program': 'b', 'promítání': 'r', 'divadlo': 'pink'}
def patch_label_coords(p):
    offset = 0
    if p.get_width() < 0:
        offset = - 0.03
    return p.get_x() + p.get_width() + offset, p.get_y() - 0.25

In [ ]:
program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T.sort_values('Zúčastnil(a) jsem se', ascending=True).plot.barh(stacked=True, figsize=(30, 50))

#### splitting program to categories

In [ ]:
col_to_category

In [ ]:
program_attendance = program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T['Zúčastnil(a) jsem se'].to_frame()
program_attendance['type'] = program_attendance.index.to_series().map(col_to_category)

sorted_attendance = program_attendance.sort_values(by='Zúčastnil(a) jsem se', ascending=True)
sorted_attendance['Zúčastnil(a) jsem se'].plot.barh(stacked=True, figsize=(30, 50), color=sorted_attendance['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.0f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
program_attendance['type'] = program_attendance.index.to_series().map(col_to_category)
program_attendance.head()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].sum().sort_values(ascending=True).plot.barh()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].mean().sort_values(ascending=True).plot.barh()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].count().sort_values(ascending=True).plot.barh()

In [ ]:
sns.violinplot(x='type', y='Zúčastnil(a) jsem se', data=program_attendance, inner='stick', bw=.3)
plt.title('návštěvnost')
plt.xticks(rotation=15)

In [ ]:
rating_to_num = {'špatné': -1, 'v pohodě': 0, 'dobré': 1}
fill_to_num = {'chci hodnotit stručně': -1, 'chci hodnotit podrobně': 1}
app_to_num = {'Ne': -1, 'Ne, nevědel sem o ní': -1, 'Ne, z jiného důvodu': -1, 'Ne, nemám podporované zařízení': -1, 'Ne, nedokázal jsem ji spustit': -1, 'Ne, nezajímá mě': -1, np.nan: 0, 'Ano': 1}
bool_to_num = {'Ne': -1, np.nan: 0, 'Ano': 1}
age_to_num = {'13-15': 1, '16-18': 2, '19-25': 3, '26 a více': 4}
    
#sns.pairplot(df_form[cols_to_plot], hue='Pohlaví')
plt.figure(figsize=(20, 20))
g = sns.pairplot(df_form[['Věk', 'Pohlaví', 'Jak podrobně chcete hodnotit?', 'Připadá vám AF téma pro zážitek na conu důležité?', 'Používali jste Animefest aplikaci?']].replace({
    'Věk': age_to_num, 'Jak podrobně chcete hodnotit?': fill_to_num, 'Připadá vám AF téma pro zážitek na conu důležité?': bool_to_num, 'Používali jste Animefest aplikaci?': app_to_num
}), hue='Pohlaví')
[plt.setp(ax.xaxis.get_label(), rotation=15) for ax in g.axes.flat]
[plt.setp(ax.yaxis.get_label(), rotation=75) for ax in g.axes.flat]


In [ ]:
program_fun_score, program_info_score = calc_scores(program_ratings, program_columns)

In [ ]:
program_ratings.head()

In [ ]:
program_fun_score.head()

In [ ]:
program_info_score.head()

In [ ]:
sorted_score = program_fun_score.sort_values(by='score', ascending=True)
sorted_score['score'].plot.barh(figsize=(30, 50), color=sorted_score['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.2f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
sorted_score = program_info_score.sort_values(by='score', ascending=True)
sorted_score['score'].plot.barh(figsize=(30, 50), color=sorted_score['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.2f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
sns.violinplot(x='type', y='score', data=program_fun_score, inner='stick', bw=.3)
plt.title('zábavnost')
plt.xticks(rotation=15)

In [ ]:
sns.violinplot(x='type', y='score', data=program_info_score, inner='stick', bw=.3)
plt.title('informovanost')
plt.xticks(rotation=15)

In [ ]:
### can't see number of them, need parallel sets

In [ ]:
{i: df_form[i].unique() for i in cols_to_plot}

In [ ]:
df_form['af_aplikace_bool'] = df_form['Používali jste Animefest aplikaci?'].replace({'Ne, nedokázal jsem ji spustit': 'Ne', 'Ne, nemám podporované zařízení': 'Ne', 'Ne, nevědel sem o ní': 'Ne', 'Ne, nezajímá mě': 'Ne', 'Ne, z jiného důvodu': 'Ne'})

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df_form['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df_form['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

place_dim = go.parcats.Dimension(
    values=df_form['Bydliště'],
    categoryorder='category ascending', label='Bydliště'
)

detail_dim = go.parcats.Dimension(
    values=df_form['Jak podrobně chcete hodnotit?'],
    categoryorder='category ascending', label='Podrobné hodnocení'
)

app_detailed_dim = go.parcats.Dimension(
    values=df_form['Používali jste Animefest aplikaci?'],
    categoryorder='category ascending', label='Používání AF aplikace'
)

app_dim = go.parcats.Dimension(
    values=df_form['Používali jste Animefest aplikaci?'].replace({'Ne, nedokázal jsem ji spustit': 'Ne', 'Ne, nemám podporované zařízení': 'Ne', 'Ne, nevědel sem o ní': 'Ne', 'Ne, nezajímá mě': 'Ne', 'Ne, z jiného důvodu': 'Ne'}),
    categoryorder='category ascending', label='Používání AF aplikace'
)


souvenirs_price_dim = go.parcats.Dimension(
    values=df_form['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

more_souvenirs_dim = go.parcats.Dimension(
    values=df_form['Uvítali byste nějaké další suvenýry?'],
    categoryorder='category ascending', label='Více suvenýrů?'
)

# not filled in detailed version
factions_dim = go.parcats.Dimension(
    values=df_form['Letošní téma a frakce'],
    categoryorder='category ascending', label='téma a frakce'
)

video_dim = go.parcats.Dimension(
    values=df_form['Úvodní/závěrečné video'],
    categoryorder='category ascending', label='Úvodní/závěrečné video'
)

points_dim = go.parcats.Dimension(
    values=df_form['Sbírání bodů'],
    categoryorder='category ascending', label='Sbírání bodů'
)

theme_dim = go.parcats.Dimension(
    values=df_form['Připadá vám AF téma pro zážitek na conu důležité?'],
    categoryorder='category ascending', label='Je AF téma důležité?'
)


# Create parcats trace
#color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
color = df_form['Úvodní/závěrečné video'].replace({'dobré': 0, 'v pohodě': 0.2, 'špatné': 0.4, 'Nevím o co jde': 0.6, np.nan: 1})
#colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=
#                                  [age_dim, gender_dim, app_dim, souvenirs_price_dim, video_dim, theme_dim],
                                   [age_dim, gender_dim, video_dim, theme_dim],
#        line={'color': color, 'colorscale': colorscale},
        line={'color': color},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
df = df_form[['Věk', 'Pohlaví', 'Kromě letošního ročníku jsem na Animefestu byl(a)', 'Připadá vám AF téma pro zážitek na conu důležité?', 'Úvodní/závěrečné video']].dropna()
df['Kromě letošního ročníku jsem na Animefestu byl(a)'] = df['Kromě letošního ročníku jsem na Animefestu byl(a)'].replace({'6x': '6x+', '7x': '6x+', '8x': '6x+', '9x': '6x+', 
                                                                                                                           '10x': '6x+', '11x': '6x+', '12x': '6x+', '13x': '6x+', '14x': '6x+'})
# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

num_visits_dim = go.parcats.Dimension(
    values=df['Kromě letošního ročníku jsem na Animefestu byl(a)'],
    categoryorder='category ascending', label='Kolikrát předtím na AF'
)

video_dim = go.parcats.Dimension(
    values=df['Úvodní/závěrečné video'],
    categoryorder='category ascending', label='Úvodní/závěrečné video'
)

theme_dim = go.parcats.Dimension(
    values=df['Připadá vám AF téma pro zážitek na conu důležité?'],
    categoryorder='category ascending', label='Je AF téma důležité?'
)


# Create parcats trace
#color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
color = df['Úvodní/závěrečné video'].replace({'dobré': 0, 'v pohodě': 0.2, 'špatné': 0.4, 'Nevím o co jde': 0.6, np.nan: 1})
#colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=
                                   [age_dim, gender_dim, num_visits_dim, video_dim, theme_dim],
#        line={'color': color, 'colorscale': colorscale},
        line={'color': color},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
df = df_form[['Věk', 'Pohlaví', 'Cena AF suvenýrů je přiměřená?']].dropna()

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

souvenirs_price_dim = go.parcats.Dimension(
    values=df['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

# Create parcats trace
color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, gender_dim, souvenirs_price_dim],
        #line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
# for souvenirs price

df = df_form[['Věk', 'Bydliště', 'Pohlaví', 'Cena vstupného je…', 'Cena AF suvenýrů je přiměřená?']].dropna()

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

place_dim = go.parcats.Dimension(
    values=df['Bydliště'],
    categoryorder='category ascending', label='Bydliště'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

ticket_price_dim = go.parcats.Dimension(
    values=df['Cena vstupného je…'],
    categoryorder='category ascending', label='Cena vstupenek'
)

souvenirs_price_dim = go.parcats.Dimension(
    values=df['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

# Create parcats trace
color = df['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, gender_dim, ticket_price_dim, souvenirs_price_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

'Dám i víc' dali jen muži 19+

### counting conditional probabilities

In [ ]:
def cond_probs(df, factors, column):
    temp_df = df.groupby(factors)[column].value_counts() / df.groupby(factors)[column].count()
    temp_df = temp_df.unstack(level=-1)
    if np.nan in temp_df.columns:
        temp_df = temp_df.drop(columns=np.nan)
    return temp_df

In [ ]:
cond_probs(df_form, 'Pohlaví', 'Cena AF suvenýrů je přiměřená?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Cena AF suvenýrů je přiměřená?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Používali jste Animefest aplikaci?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Úvodní/závěrečné video')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Připadá vám AF téma pro zážitek na conu důležité?')

In [ ]:
cond_probs(df_form, ['Připadá vám AF téma pro zážitek na conu důležité?'], 'Úvodní/závěrečné video')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví', 'Úvodní/závěrečné video'], 'Připadá vám AF téma pro zážitek na conu důležité?')

In [ ]:
cols_to_plot

In [ ]:
df_form.head()

In [ ]:
people_ratings = df_form[program_columns_ratings].apply(lambda x: x.value_counts(), axis=1)
people_ratings = people_ratings.dropna(axis='rows', how='all', subset=['dobré', 'v pohodě', 'špatné']).fillna(0)
people_ratings.head()

In [ ]:
people_score = (people_ratings['dobré'] * 1 + people_ratings['v pohodě'] * 0 + people_ratings['špatné'] * -1) / people_ratings.sum(axis=1)
people_score.head()

In [ ]:
people_score.plot.hist(bins=200, figsize=(20, 10))

In [ ]:
#people_score.plot.hist(bins=200, figsize=(20, 10))
#sns.violinplot(x='type', y='score', data=people_score, inner='stick', bw=.3)
sns.violinplot(data=people_score.to_frame(), inner='stick', bw=.2)
plt.title('informovanost')
plt.xticks(rotation=15)

In [ ]:
#people_score.plot.hist(bins=200, )
#sns.violinplot(x='type', y='score', data=people_score, inner='stick', bw=.3)
data_to_plot = pd.merge(df_form[['Věk', 'Pohlaví']], people_score.to_frame(name='score'), left_index=True, right_index=True).sort_values(by='Věk')
plt.figure(figsize=(20, 10))
sns.violinplot(x='Věk', y='score', hue='Pohlaví', data=data_to_plot, inner='stick', bw=.3)
plt.title('průměrné hodnocení')
plt.xticks(rotation=15)

### analysis of open answers

In [ ]:
from urllib import request

data = request.urlopen('https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.txt')
all_stopwords = [line.decode('utf-8').replace('\n', '') for line in data]
# todo: split them correctly and for vocalulary calculation use only emotes filtering
emotes_stopwords = [':d', ':', '\?', ';', ':-d', '\^\^', ':dd', 'xd', ':3', '=d', ':p',  ':/', ':>', '\+', 'o_o', 'd:', '>', ':\'d', ':"d', '\n', ':dd', ':ddd', ':dddd', 'x', 'xp']
my_stopwords = ['no', '', 'jo', 'sa', 'the', '=d', 'fakt', 'celkem', 'sem', 'ja', 'som', 'ako', 'of', 'nekdo', 'bych', 'sou', '2', 'jj', 'is', 'mam', '3', '\n']
all_stopwords += my_stopwords
all_stopwords += emotes_stopwords

def messages_to_words_hist(mess_series):
    mess_series = mess_series.fillna('')
    all_words = mess_series.str.split(' ', expand=True).stack().reset_index(drop=True, level=1)
    words_historgram = all_words.str.lower().str.strip('.,?-–()').value_counts()
    stopwords = list(set(all_stopwords).intersection(set(words_historgram.index)))
    words_historgram_cleaned = words_historgram.drop(stopwords)
    return words_historgram_cleaned


In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'].str.contains('přednášky', regex=False)]['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co vás letos nejvíc zklamalo / nelíbilo se vám?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'].str.contains('přednášky', regex=False)]['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste měli problémy?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='S čím jste měli problémy?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['Uvítali byste nějaké další suvenýry?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Uvítali byste nějaké další suvenýry?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Uvítali byste nějaké další suvenýry?'] != '']['Uvítali byste nějaké další suvenýry?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Hodnocení a poznámky ke grafice'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Hodnocení a poznámky ke grafice')
plt.xticks(rotation=30)

In [ ]:
[i for i in open_answers if 'Komentář' not in i]

In [ ]:
best_mess = messages_to_words_hist(df_form['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'] != '']['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'].tolist())

### individual lectures/workshops analysis

In [ ]:
# todo: udělat violin plot průměrných hodnocení pro jednotlivé lidi a různé demografie. A kouknout na množství překryvů mezi hodnoceným programem časově, zda to šlo stihnout

In [ ]:
df_timetable[df_timetable['scheduleEntryId'].isin(df_timetable['scheduleEntryId'].value_counts()[df_timetable['scheduleEntryId'].value_counts() > 1].index)]

In [ ]:
pd.Series(df_timetable[df_timetable['scheduleEntryId'].isin(df_timetable['scheduleEntryId'].value_counts()[df_timetable['scheduleEntryId'].value_counts() > 1].index)]['name'].unique())


In [ ]:
pd.Series(program_columns.str.replace(info_suffix, '', regex=False).str.replace(fun_suffix, '', regex=False).unique())

### temporal and spatial analysis of occupancy rate

In [ ]:
program_attendance.head()

In [ ]:
df_timetable

In [ ]:
attendance_with_loc = pd.merge(program_attendance, df_timetable, left_index=True, right_on='name')
attendance_with_loc['occupancy'] = attendance_with_loc['Zúčastnil(a) jsem se'] / attendance_with_loc['capacity']
# some are over 100%, setting them to 100%, but further inspection is needed
attendance_with_loc.loc[attendance_with_loc['occupancy'] > 1, 'occupancy'] = 1
#attendance_with_loc['from_date'] = attendance_with_loc['from'].dt.date
attendance_with_loc['from_date'] = attendance_with_loc['from'].dt.day
#attendance_with_loc['from_time'] = attendance_with_loc['from'].dt.time.astype(str).map(lambda x: x[:5])
attendance_with_loc['from_time'] = attendance_with_loc['from'].dt.hour
attendance_with_loc = attendance_with_loc[attendance_with_loc['note'] != 'stream']
attendance_with_loc = attendance_with_loc[attendance_with_loc['type'] != 'promítání']
attendance_with_loc.head()

In [ ]:
sorted_occupancy = attendance_with_loc.sort_values(by='occupancy', ascending=True)
sorted_occupancy = sorted_occupancy.set_index('name')
sorted_occupancy = sorted_occupancy.loc[~sorted_occupancy.index.duplicated(keep='first')]
sorted_occupancy['occupancy'].plot.barh(stacked=True, figsize=(30, 60), color=sorted_occupancy['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("{:,.1%}".format(p.get_width()), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')
plt.gca().set_xticklabels(['{:,.1%}'.format(x) for x in plt.gca().get_xticks()])
plt.tight_layout()
plt.savefig('occupancy-per-lecture.png')

In [ ]:
sns.violinplot(x='location_name', y='occupancy', data=attendance_with_loc, inner='stick', bw=.3)
plt.title('obsazenost místností')
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
plt.xticks(rotation=40)
plt.tight_layout()
plt.savefig('occupancy-per-room.png')

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x='from_time', y='occupancy', data=attendance_with_loc, inner='stick', bw=.7, order=sorted(attendance_with_loc['from_time'].unique()))
plt.title('obsazenost podle času [hodiny]')
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
plt.tight_layout()
plt.savefig('occupancy-per-time.png')

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x='from_time', y='Zúčastnil(a) jsem se', data=attendance_with_loc, inner='stick', bw=.3, order=sorted(attendance_with_loc['from_time'].unique()))
plt.title('účast podle času [hodiny]')
plt.tight_layout()
plt.savefig('attendance-per-time.png')

In [ ]:
sns.violinplot(x='from_date', y='occupancy', data=attendance_with_loc, inner='stick', bw=.3)
plt.title('obsazenost podle dne')
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
plt.tight_layout()
plt.savefig('occupancy-per-date.png')

In [ ]:
sns.violinplot(x='from_date', y='Zúčastnil(a) jsem se', data=attendance_with_loc, inner='stick', bw=.3)
plt.title('účast podle dne')
plt.tight_layout()
plt.savefig('attendance-per-date.png')

In [ ]:
plt.figure(figsize=(15,10))
attendance_with_loc.groupby('from_time')['occupancy'].mean().plot.bar()
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
plt.title('účast podle času [hodiny]')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('occupancy-mean-per-time.png')

In [ ]:
plt.figure(figsize=(15,10))
attendance_with_loc.groupby('from_time')['Zúčastnil(a) jsem se'].sum().plot.bar()
plt.title('účast podle času [hodiny]')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('attendance-sum-per-time.png')

In [ ]:
attendance_with_loc[attendance_with_loc['from_time'] == 18].sort_values(by='Zúčastnil(a) jsem se', ascending=False)

In [ ]:
plt.figure(figsize=(15,10))
attendance_with_loc.groupby('from_date')['Zúčastnil(a) jsem se'].sum().plot.bar()
plt.title('účast podle dne')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('attendance-sum-per-date.png')